## 🟢 **SAR Padawans: Understanding Array Sampling and Beamforming**  🌱

Let's revisit the **sampling of our array**, but now from the **receiving perspective**. Your goal is to **simulate an echo signal** for a phased array and apply **beamforming** to correctly identify the **azimuth angle of the target echo.**

### **🔹 Task 1: Debug and Complete the Beamforming Code**  
- Use the provided **beamforming code** and **fix any issues** in how the **phase shifts** are computed for each element.  
- The final check: Ensure that **the target appears at the correct angle** in the output.  

### **🔹 Task 2: Experiment with Undersampling and Angle Spacing**  
- Increase the **element spacing** of your array to be **greater than the Nyquist limit**.  
- What do you **observe**? Can you **explain** why? 🤔

---

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

### Let's first define the simulation parameters

In [ ]:
# First simulate the element signals:
number_of_elements = 10 # array elements
wavelength = 0.03 # m
target_angle_degrees = 12 # azimuth angle in degrees
target_angle = np.radians(target_angle_degrees)
target_amplitude = 1.0
element_spacing = 4* wavelength / 2 # adjust this value later

### Here are some template functions to help you get started. Watch out for bugs! 🐛

In [ ]:
# Computes the received signal phasors for each antenna element from a target at given azimuth angle
def compute_element_phasors(element_positions_x, wavelength, target_angle, target_amplitude):
    """
    Compute the phasors for antenna array elements based on the waveront direction
    """
    k = 2 * np.pi / wavelength # wavenumber
    # Path length difference is element_position*sin(azimuth_angle)
    element_path_length_difference = element_positions_x * np.sin(target_angle)
    target_phasors = target_amplitude * np.exp(1j * k * element_path_length_difference)
    return target_phasors

# Function to combine the received element echoes into an angular profile
def compute_beam_output(element_phasors, angle_min_degrees, angle_max_degrees, angle_step, positions, wavelength):
    """
    Perform an angular scan using the received element signals, to obtain the received wave height vs. angle
    """
    nb_angles = int((angle_max_degrees - angle_min_degrees) / angle_step) # Number of beamforming scan angles
    scan_angles = np.radians(np.linspace(angle_min_degrees, angle_max_degrees, nb_angles))  # Beamforming angles
    beam_output = np.zeros((len(scan_angles),), dtype=np.complex64)

    for i, scan_angle in enumerate(scan_angles):
        # Calculate and compensate the phase shift for this angle
        scan_phase_shift = 2 * np.pi * np.sin(scan_angle) * element_positions_x / wavelength
        scan_phase_shifts = np.exp(-1j * scan_phase_shift) # Phase correction corresponding to this scan angle
        beam_output[i] = np.sum(element_phasors * scan_phase_shifts) # apply phase correction and sum phasors together

    # Convert beam output to magnitude for visualization
    beam_output_magnitude = np.abs(beam_output)
    beam_output_magnitude /= np.max(beam_output_magnitude)  # Normalize for visualization

    return scan_angles, beam_output_magnitude    

### Let's simulate the phasors for each antenna element

In [ ]:
# x-position of each element
element_positions_x = np.linspace(-(number_of_elements-1)/2 * element_spacing, (number_of_elements-1)/2 * element_spacing, number_of_elements)
# simulate phasors representing the wave received by each element
element_phasor_data = compute_element_phasors(element_positions_x, wavelength, target_angle, target_amplitude)

### Now your task is to fix the beamforming calculations in `compute_beam_output()` 🐛🐞🦟

In [ ]:
# You can adjust the limits and the spacing of the scanned angle interval
angle_min_deg = -60  # Minimum scan angle (degrees)
angle_max_deg = 60   # Maximum scan angle (degrees)
angle_step = 1   # Angle spacing (degrees)
scan_angles, beam_output = compute_beam_output(element_phasor_data, angle_min_deg, angle_max_deg, angle_step, element_positions_x, wavelength)

### Then plot the results 📈

In [ ]:
# Create a single figure with one subplot
fig, ax_beam = plt.subplots(figsize=(8, 5))
# Plot the beamforming response
ax_beam.plot(np.degrees(scan_angles), beam_output, 'k-', label="Beam Response")
ax_beam.axvline(target_angle_degrees, color='r', linestyle='--', label=f"Target ({target_angle_degrees:.1f}°)")
# Labels and formatting
ax_beam.set_xlabel("Azimuth angle (degrees)")
ax_beam.set_ylabel("Beamforming Response Magnitude")
ax_beam.set_title("Beamforming Output")
ax_beam.legend()
ax_beam.grid(True)
plt.show()

### Once your beamforming code works, start experimenting with the element spacing. What do you observe when the spacing is larger than half a wavelength?

### ⚠️ Aliasing Alert: When Spacing is Too Wide!

If we use a **spacing larger than half a wavelength**, we encounter **aliasing**! 🔄 This happens when the **phase shift** between neighboring elements exceeds **half a cycle**, causing the phase to become **ambiguous**. ❓⚡

---

### 📡 Grating Lobes and Ambiguous Returns

For **arrays**, this leads to the formation of **grating lobes**. In **receive beamforming**, this means that at certain angles, the **beam pattern repeats itself** because the phase differences are **wrapped**. 🔁 This results in **ambiguous returns**, where signals from different directions appear indistinguishable.

It’s the **same principle** that causes **azimuth ambiguities** in **SAR images**. 🛰️🎯 So, careful element spacing is essential to avoid these unwanted duplicates!


### 🎯 Calculating the Maximum Unambiguous Angle

We can calculate the **maximum unambiguous angle** at which a target can be located **without causing phase aliasing**. 

For this, the **path length difference** between elements must be **less than half a wavelength**, leading to the condition:

$$
\sin(\theta_{\textrm{max}}) \cdot \Delta x = \frac{\lambda}{2}
$$

Solving for the angle gives:

$$
\theta_{\textrm{max}} = \arcsin\left(\frac{\lambda}{2\Delta x}\right)
$$

This defines the **maximum angle** where the phase difference remains unambiguous. Exceeding this limit introduces **aliasing** and results in ambiguous measurements. ⚠️

The angle can be either **positive or negative**, so multiplying the **maximum unambiguous angle by two** gives the **distance between the grating lobes**. This defines the **angular interval** at which the target response **repeats itself** due to aliasing. 🔄🎯


In [ ]:
theta_max = np.degrees(np.asin(wavelength / (2*element_spacing)))
print("Maximum unambiguous azimuth angle is (plus minus):", theta_max, "degrees")
print("Grating lobes will appear:", 2*theta_max, "degrees apart")